In [5]:
import pandas as pd
import numpy as np
import math
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import jieba
from tqdm import tqdm
import csv
import pickle

In [ ]:
def read(file_name):
    urls = []
    commend = []
    colors = []
    size = []
    url_title = []
    with open(file_name,'r',encoding = 'utf-8') as myFile:  
        lines=csv.reader(myFile)  
        for line in lines:
            if(len(line) == 1):
                url = line[0]
            if(len(line)>1):
                urls.append(url)
                commend.append(line[2])
                colors.append(line[3])
                size.append(line[4])
                url_title.append(line[6]) 
    return pd.DataFrame({'url': urls, 'comment': commend, 'colors': colors, 'size':size, 'url_title': url_title})

def remove_CSVnull(in_file, out_file):
    fi=open(in_file,'rb')
    data=fi.read()
    fi.close()

    fo=open(out_file,'wb')
    fo.write(data.replace(b'\x00',b''))
    fo.close()

def get_url_attr_commends_useinfo(commfile, urlfile):
    comments = read(commfile)
    url_title = pd.read_csv(urlfile)
    if 'more_attrs' not in url_title.columns:
        url_title['more_attrs'] = None
    url_attr_comments_useinfo = comments.merge(url_title, on='url', how='left')
    url_attr_comments_useinfo = url_attr_comments_useinfo[['url', 'product_name', 'price', 'attrs', 'price2', 'more_attrs', 'comment', 'colors','size', 'url_title']]
    return url_attr_comments_useinfo

# 停用词
def get_stopwords():
    with open('./stopword.csv', 'r', encoding='utf-8') as f:
        stopwords = f.read()
    return stopwords

def pickleLoad(path):
    obj = None
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

def pickleDump(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

In [ ]:
# 去除csv中的null
# remove_CSVnull('./input/ytj-result.csv', './input/ytj-result_new.csv')
# remove_CSVnull('./input/qbb-result.csv', './input/qbb-result_new.csv')
# remove_CSVnull('./input/tsj-result.csv', './input/tsj-result_new.csv')

In [ ]:
# # 将评论与商品的属性拼接起来
# ytj_url_comments = get_url_attr_commends_useinfo('./input/ytj-result_new.csv', './input/ytj.csv')
# qbb_url_comments = get_url_attr_commends_useinfo('./input/qbb-result_new.csv', './input/qbb.csv')
# bjb_url_comments = get_url_attr_commends_useinfo('./input/bjb-result_new.csv', './input/bjb.csv')
# tsj_url_comments = get_url_attr_commends_useinfo('./input/tsj-result_new.csv', './input/tsj.csv')

# # 合并所有的数据
# all_url_coments = pd.concat([ytj_url_comments, qbb_url_comments, bjb_url_comments, tsj_url_comments]).reset_index(drop=True)
# # 过滤掉不是电脑的数据
# key_words = ['显卡', '系统', '硬盘', 'Windows', '内存','i5', '笔记本电脑', '台式电脑', '台式机']
# def is_cp(x):
#     global key_words
#     for key in key_words:
#         if key in x['attrs']:
#             return True
#     return False

# cp_relative = all_url_coments.apply(is_cp, axis=1)
# all_url_coments = all_url_coments[cp_relative].reset_index(drop=True)
# all_url_coments.to_csv('./input/all_url_coments_pc.csv', index=False)

# 模型

In [3]:
all_url_coments = pd.read_csv('./input/all_url_coments_pc.csv')

D:\ProgramFile\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
url_df = pd.DataFrame({'url': list(set(all_url_coments.url.values.tolist()))})
url_df.to_csv('./zhiwei_url.csv', index=False)

In [ ]:
# 过滤掉name不涉及游戏的数据
key_words = ['游戏']
def is_cp(x):
    global key_words
    for key in key_words:
        if key in x['product_name']:
            return True
    return False

cp_relative = all_url_coments.apply(is_cp, axis=1)
all_url_coments = all_url_coments[cp_relative].reset_index(drop=True)

In [ ]:
# 去掉比较短的评论(20字以下的)
long_comm = all_url_coments['comment'].str.len() >= 20
long_url_comments = all_url_coments[long_comm].reset_index(drop=True)
long_url_comments.to_csv('./input/long_url_comments.csv', index=False)

In [ ]:
# # 取出所有的评论
# comments = long_url_comments.sample(random_state=2018, n=2000)['product_name'].values.tolist()

# 获取title
comments = list(set(long_url_comments['product_name'].values.tolist()))
len(comments)

In [ ]:
# 分词
comments_words = []
for comm in tqdm(comments):
    seg_list = jieba.cut(comm)  # 默认是精确模式
    comments_words.append(' '.join(seg_list))
    
    
# 去停用词
stopwords = get_stopwords()
new_comments_words = []
for comm_words in tqdm(comments_words):
    new_comm_words = []
    for word in comm_words.strip().split():
        if word not in stopwords:
            new_comm_words.append(word)
    new_comments_words.append(' '.join(new_comm_words))

# 利用sklearn countvectorizer 或取文档的bow表示(稀疏矩阵)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(new_comments_words) #(4854, 3021)
# pickleDump(vectorizer, './tf_vectorizer.pickle')
X.shape


In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
# 训练LDA
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=5, 
                                max_iter=1000, 
                                learning_method='batch',
                                verbose=1,
                                random_state=2018)
lda.fit(X)

@评论
物流
售后
外观
性价比
开机速度、流畅
性能
屏幕
散热

In [ ]:
print("\nTopics in LDA model:")
n_top_words = 10
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
# 每篇文档的主题分布
doc_topic_dist = lda.transform(X)

In [ ]:
doc_topic_dist[]